Prdidiction


In [0]:
import numpy as np
import pandas as pd


df = pd.read_csv("train.csv")
train = pd.read_csv(f"train.csv", low_memory=False, 
                    parse_dates=['date'], index_col=['date'])
test = pd.read_csv(f"test.csv", low_memory=False, 
                   parse_dates=['date'], index_col=['date'])

In [0]:
#resolved_score = training_df[training_df['essay_set'] == essay_set]['domain1_score']
train = train[train["item"]<15]
train = train[ train["store"] < 2]
test = test[test["item"]<15]
test = test[test["store"] < 2]

In [0]:
def expand_df(df):
    data = df.copy()
    data['day'] = data.index.day
    data['month'] = data.index.month
    data['year'] = data.index.year
    data['dayofweek'] = data.index.dayofweek
    return data

In [0]:
data = expand_df(train)
test_data = expand_df(test)

In [0]:
print(data)

            store  item  sales  day  month  year  dayofweek
date                                                       
2013-01-01      1     1     13    1      1  2013          1
2013-01-02      1     1     11    2      1  2013          2
2013-01-03      1     1     14    3      1  2013          3
2013-01-04      1     1     13    4      1  2013          4
2013-01-05      1     1     10    5      1  2013          5
...           ...   ...    ...  ...    ...   ...        ...
2017-12-27      1    14     42   27     12  2017          2
2017-12-28      1    14     51   28     12  2017          3
2017-12-29      1    14     42   29     12  2017          4
2017-12-30      1    14     57   30     12  2017          5
2017-12-31      1    14     52   31     12  2017          6

[25564 rows x 7 columns]


In [0]:
data_y = []
for i in data.sales :
   if i > 65:
     data_y.append(3)
   elif i > 48:
     data_y.append(2)
   elif i > 30:
     data_y.append(1)
   else:
     data_y.append(0)

In [0]:
for j in range(4):
  c=0
  for i in data_y:
    if i == j:
      c+=1
  print(j,c)
len(data_y)

0 6674
1 5964
2 6572
3 6354


25564

In [0]:
data_y = np.array(data_y)

In [0]:
def num_data(data):
  E = zip(data.item,data.day,data.month,data.dayofweek)

  X = []

  for i,j,k,l in E :
    X.append([i,j,k,l])
  X = np.array(X)
  return X

train_x = num_data(data)

#train_y = np.array(data.sales)
train_y = data_y
test_x = num_data(test_data) 

#from sklearn import preprocessing
#X_scaled = preprocessing.scale(train_x)
#Y_scaled = preprocessing.scale(train_y) 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x,train_y, test_size=0.2)

In [0]:
print(len(X_train))
print()

20451



In [0]:
from sklearn import svm
regr = svm.SVC()
regr.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
predict=regr.predict(X_test)

In [0]:
print(predict)

[0 1 1 ... 3 2 2]


In [0]:
import sklearn
sklearn.metrics.mean_squared_error(y_test,predict)

0.8036377860355955

Shortest Path algorithm for Dynamic routing

In [0]:
# here in this section we combine both of the work
#     1) take box for delivery
#     2) Put box at the blank place
#     

In [0]:
# WE are useing famouse A-start algorithm for predicting the shortest path from current location

import numpy as np

class Node:
    """
        A node class for A* Pathfinding
        parent is parent of the current Node
        position is current position of the Node in the maze
        g is cost from start to current Node
        h is heuristic based estimated cost for current Node to end Node
        f is total cost of present node i.e. :  f = g + h
    """

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.position == other.position

#This function return the path of the search
def return_path(current_node,maze):
    path = []
    no_rows, no_columns = np.shape(maze)
    # here we create the initialized result maze with -1 in every position
    result = [[-1 for i in range(no_columns)] for j in range(no_rows)]
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    # Return reversed path as we need to show from start to end path
    path = path[::-1]
    start_value = 0
    # we update the path of start to end found by A-star serch with every step incremented by 1
    for i in range(len(path)):
        result[path[i][0]][path[i][1]] = start_value
        start_value += 1
    return result


def search(maze, cost, start, end):
    """
        Returns a list of tuples as a path from the given start to the given end in the given maze
        :param maze:
        :param cost
        :param start:
        :param end:
        :return:
    """

    # Create start and end node with initized values for g, h and f
    start_node = Node(None, tuple(start))
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, tuple(end))
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both yet_to_visit and visited list
    # in this list we will put all node that are yet_to_visit for exploration. 
    # From here we will find the lowest cost node to expand next
    yet_to_visit_list = []  
    # in this list we will put all node those already explored so that we don't explore it again
    visited_list = [] 
    
    # Add the start node
    yet_to_visit_list.append(start_node)
    
    # Adding a stop condition. This is to avoid any infinite loop and stop 
    # execution after some reasonable number of steps
    outer_iterations = 0
    max_iterations = (len(maze) // 2) ** 10

    # what squares do we search . serarch movement is left-right-top-bottom 
    #(4 movements) from every positon

    move  =  [[-1, 0 ], # go up
              [ 0, -1], # go left
              [ 1, 0 ], # go down
              [ 0, 1 ]] # go right


    """
        1) We first get the current node by comparing all f cost and selecting the lowest cost node for further expansion
        2) Check max iteration reached or not . Set a message and stop execution
        3) Remove the selected node from yet_to_visit list and add this node to visited list
        4) Perofmr Goal test and return the path else perform below steps
        5) For selected node find out all children (use move to find children)
            a) get the current postion for the selected node (this becomes parent node for the children)
            b) check if a valid position exist (boundary will make few nodes invalid)
            c) if any node is a wall then ignore that
            d) add to valid children node list for the selected parent
            
            For all the children node
                a) if child in visited list then ignore it and try next node
                b) calculate child node g, h and f values
                c) if child in yet_to_visit list then ignore it
                d) else move the child to yet_to_visit list
    """
    #find maze has got how many rows and columns 
    no_rows, no_columns = np.shape(maze)
    
    # Loop until you find the end
    
    while len(yet_to_visit_list) > 0:
        
        # Every time any node is referred from yet_to_visit list, counter of limit operation incremented
        outer_iterations += 1    

        
        # Get the current node
        current_node = yet_to_visit_list[0]
        current_index = 0
        for index, item in enumerate(yet_to_visit_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index
                
        # if we hit this point return the path such as it may be no solution or 
        # computation cost is too high
        if outer_iterations > max_iterations:
            print ("giving up on pathfinding too many iterations")
            return return_path(current_node,maze)

        # Pop current node out off yet_to_visit list, add to visited list
        yet_to_visit_list.pop(current_index)
        visited_list.append(current_node)

        # test if goal is reached or not, if yes then return the path
        if current_node == end_node:
            return return_path(current_node,maze)

        # Generate children from all adjacent squares
        children = []

        for new_position in move: 

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range (check if within maze boundary)
            if (node_position[0] > (no_rows - 1) or 
                node_position[0] < 0 or 
                node_position[1] > (no_columns -1) or 
                node_position[1] < 0):
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:
            
            # Child is on the visited list (search entire visited list)
            if len([visited_child for visited_child in visited_list if visited_child == child]) > 0:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + cost
            ## Heuristic costs calculated here, this is using eucledian distance
            child.h = (((child.position[0] - end_node.position[0]) ** 2) + 
                       ((child.position[1] - end_node.position[1]) ** 2)) 

            child.f = child.g + child.h

            # Child is already in the yet_to_visit list and g cost is already lower
            if len([i for i in yet_to_visit_list if child == i and child.g > i.g]) > 0:
                continue

            # Add the child to the yet_to_visit list
            yet_to_visit_list.append(child)


if __name__ == '__main__':

    maze = [[0, 0, 0, 0, 0, 0,0], # we are assuming that our Warehouse is 7x7 matrix, where '0' indicates the path  
            [0, 1, 0, 1, 0, 1,0], # while 1 is the cupbord that robat can not cross
            [0, 1, 0, 1, 0, 1,0], #==>"Through this design we have flaxibility to adopt any warehouse design and we can implimate SP routing "
            [0, 0, 0, 0, 0, 0,0],
            [0, 1, 0, 1, 0, 1,0],
            [0, 1, 0, 1, 0, 1,0],
            [0, 0, 0, 0, 0, 0,0]]
    
    start = [0, 0] # starting position
    end = [4,5] # ending position
    cost = 1 # cost per movement

    path = search(maze,cost, start, end)
    print(path)

None


In [0]:
'''
following code with illustarte how robot will find the path
Assume that blank_place containts all the blank places where robot has to visit
'''

In [0]:
blank_places = [ [1,0], [3,0], [2,2], [4,2], [4,4], [3,4]]
blank_places.append([0,0])
start = [0,0]
end = blank_places[0]

for i in range(len(blank_places)) :
  
  path = search(maze,cost, start, end)
  if i == len(blank_places)-1 :
    break
  print("start : ",start)
  print("End : ",end)
  start = blank_places[i]
  end = blank_places[i+1]
  print('\n'.join([''.join(["{:" ">3d}".format(item) for item in row]) 
      for row in path]))
  #print(start)
  #print(end) 

'''
OUTPUT explaination :-
  here we can see that how robot will travers from start from end,
  all the integer from 0,1,2,3,... shows that how robot will travers and its distance and direction of traversal
'''

start :  [0, 0]
End :  [1, 0]
  0 -1 -1 -1 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
start :  [1, 0]
End :  [3, 0]
 -1 -1 -1 -1 -1 -1 -1
  0 -1 -1 -1 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1
  2 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
start :  [3, 0]
End :  [2, 2]
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1  3 -1 -1 -1 -1
  0  1  2 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
start :  [2, 2]
End :  [4, 2]
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1  0 -1 -1 -1 -1
 -1 -1  1 -1 -1 -1 -1
 -1 -1  2 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
start :  [4, 2]
End :  [4, 4]
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1  1  2  3 -1 -1
 -1 -1  0 -1  4 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
start :  [4, 4]
End :  [3, 4]
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1

'\nOUTPUT explaination :-\n  here we can see that how robot will travers from start from end,\n  all the integer from 0,1,2,3,... shows that how robot will travers and its distance and direction of traversal\n'

In [0]:
 

print('\n'.join([''.join(["{:" ">3d}".format(item) for item in row]) 
      for row in path]))
print(" ")
print('\n'.join([''.join(["{:" ">3d}".format(item) for item in row]) 
      for row in path]))



In [0]:
#input list contains the loaction of box that is arrive at warehous;
# this shows where to put that box
# and have to place at their location
Input = [[4,4],[1,4],[1,2],[1,0],[3,2],[5,4]]

#blank_places describes number of empty place in the warehouse 
blank_places = [[4,4],[3,4],[1,0],[5,4],[6,6],[2,4]]



#request is the list of all the location of boxes that are requested to the robat
request = [[1,2],[3,4],[5,2],[6,4],[6,2],[1,4]]


In [0]:
# now we will check if we has a input according to the blank_places we will load them into the our Robot
def common_member(a, b): 
    nt1 = map(tuple, a)
    nt2 = map(tuple, b)

    st1 = set(nt1)
    st2 = set(nt2)
    c =list(st1.intersection(st2))
    d =[]
    for i in c :
      d.append(list(i))
    return d



#now we calculate the total boxes that puted in the place at ware houcse

total_put = common_member(Input,blank_places) # this statement will find the all the boxes that are arrived at warehouse and has blank places in ware house
total_put = common_member(Input,request) + total_put # this statment include position of boxes that are requested by customer and also Input in warehouse

In [0]:
#now we append flag got the location so that robate can destinguish between boxes wether they are location of put the box or taking the box
for i in total_put: 
    i.append(1)
for j in request :
    j.append(0)




In [0]:
#now Final Input to the robot will be
final = request + total_put
final.sort()
final.append([0,0,0])

In [0]:
print(final)
print((len(final)))

[[1, 0, 1], [1, 2, 0], [1, 2, 1], [1, 4, 0], [1, 4, 1], [3, 4, 0], [4, 4, 1], [5, 2, 0], [5, 4, 1], [6, 2, 0], [6, 4, 0], [0, 0, 0]]
12


In [0]:
start = [0,0]
end = [final[0][0],final[0][1]]

for i in range(len(final)) :

  print(start)
  print(end)
  path = search(maze,cost, start, end)
  print('\n'.join([''.join(["{:" ">3d}".format(item) for item in row]) 
      for row in path]))
  if i == len(final)-1:
    break
  start = [final[i][0],final[i][1]]
  end = [final[i+1][0],final[i+1][1]]
  if final[i][2] :
    print("put the box")
  else:
    print("take the box")
  print("------------------------------------------------------------")
  
  
  

[0, 0]
[1, 0]
  0 -1 -1 -1 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
put the box
------------------------------------------------------------
[1, 0]
[1, 2]
  1  2  3 -1 -1 -1 -1
  0 -1  4 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
take the box
------------------------------------------------------------
[1, 2]
[1, 2]
 -1 -1 -1 -1 -1 -1 -1
 -1 -1  0 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
put the box
------------------------------------------------------------
[1, 2]
[1, 4]
 -1 -1  1  2  3 -1 -1
 -1 -1  0 -1  4 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1
take the box
------------------------------------------------------------
[1, 4]
[1, 4]
 -1 -1 -1 -1 -1 -1 -

Decision Tree


In [0]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
from pydot import graph_from_dot_data
import pandas as pd
import numpy as np

In [0]:
import pandas as pd
training_df1 = pd.read_csv("data_tree(1).tsv", delimiter='\t', encoding = "ISO-8859-1")
dist1 = training_df1['dist']
time1 = training_df1['time']
mod1 = training_df1['mode']
X = []

Y = mod1
for i in range(len(dist1)):
  X.append([dist1[i],time1[i]])

In [0]:
print(training_df1)

In [0]:
y = pd.get_dummies(Y)
X = np.array(X)
Y = np.array(y)

In [0]:
Y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [0]:
X_train

In [0]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
X_test

In [0]:
# now every box has delivery distance in KM and Time to delivery to it
# here
#   [1,0,0] = Truck
#   [0,1,0] = Airways
#   [0,0,1] = Train



In [0]:
#---------------|--Time in hour
Address =[[1200,3]]
#----------^---Distance in KM

In [0]:
y_pred = dt.predict(Address)
y_pred

array([[0, 1, 0]], dtype=uint8)

In [0]:
'''
species = np.array(y_test).argmax(axis=1)
predictions = np.array(y_pred).argmax(axis=1)
confusion_matrix(species, predictions)
'''

Prediction


In [0]:
df = pd.read_csv("train.csv")
train = pd.read_csv(f"train.csv", low_memory=False, 
                    parse_dates=['date'], index_col=['date'])
test = pd.read_csv(f"test.csv", low_memory=False, 
                   parse_dates=['date'], index_col=['date'])

In [0]:
#df = pd.read_csv("train.csv")
data = train.copy()
display(data)
data['day'] = data.index.day
data['month'] = data.index.month
data['year'] = data.index.year
data['dayofweek'] = data.index.dayofweek

In [0]:
test_data = test.copy()
display(test_data)
test_data['day'] = test_data.index.day
test_data['month'] = test_data.index.month
test_data['year'] = test_data.index.year
test_data['dayofweek'] = test_data.index.dayofweek

In [0]:
display(data)

In [0]:
def num_data(data)
  E = zip(data.item,data.day,data.month,data.dayofweek)

  X = []

  for i,j,k,l in E :
    X.append([i,j,k,l])
  X = np.array(X)
  Y = np.array(data.sales)
  return X,Y



In [0]:
 test_x = num_data(test_data)

In [0]:
from sklearn import svm
regr = svm.SVR()
regr.fit(X, Y)

In [0]:
regr.predict()

In [0]:
29,46,69

913000

In [0]:
import matplotlib.pyplot as plt 
   
# plotting the points  
plt.plot(x, y) 
plt.show()